## Likelihood comparison
Compare the likelihood and its components between the four candidate analyses (with or without the proposed extensions).

In [ ]:
from emutools.calibration import get_like_components, plot_like_components_by_analysis
from aust_covid.constants import PRIMARY_ANALYSIS, set_project_base_path

project_paths = set_project_base_path("../")
SUPPLEMENT_PATH = project_paths["SUPPLEMENT_PATH"]
RUNS_PATH = project_paths["RUNS_PATH"]

In [ ]:
requests = {
    "loglikelihood": (-28.0, -12.0, "total likelihood"),
    "ll_adult_seropos_prop": (-3.0, 2.0, "seroprevalence contribution"),
    "ll_deaths_ma": (-10.0, -3.0, "deaths contribution"),
    "ll_notifications_ma": (-17.0, -10.0, "cases contribution"),
}
like_outputs = get_like_components(requests.keys())
plot_like_components_by_analysis(like_outputs, "kdeplot", plot_requests=requests, alpha=0.1, linewidth=1.5);

Likelihood contribution statements made in manuscript.

In [ ]:
like_outputs["ll_adult_seropos_prop"].median()

In [ ]:
like_outputs["ll_notifications_ma"].median()

In [ ]:
like_outputs["ll_deaths_ma"][PRIMARY_ANALYSIS].median()

In [ ]:
violin_comparison_fig = plot_like_components_by_analysis(like_outputs, "violinplot", requests);

In [ ]:
plot_like_components_by_analysis(like_outputs, "histplot", plot_requests=requests);